In [1]:
!pip install -qq mediapipe --user

1.기본 얼굴 검출

In [2]:
import cv2
import mediapipe as mp

# MediaPipe 얼굴 감지 객체와 그리기 객체 초기화
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

# 웹캠으로부터 비디오 캡처 시작
cap = cv2.VideoCapture(0)

# 얼굴 감지 모델 설정
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("웹캠에서 이미지를 읽어오는 데 실패했습니다.")
            # 비디오 캡처가 실패한 경우, 다음 프레임을 계속 시도합니다.
            continue

        # BGR 이미지를 RGB로 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # # 성능 향상을 위해 이미지 쓰기 불가능으로 설정
        # image.flags.writeable = False
        
        # 이미지에서 얼굴 감지
        results = face_detection.process(image)

        # # 이미지를 다시 BGR로 변환하여 그리기 작업 준비
        # image.flags.writeable = True
        
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 감지된 얼굴에 대한 정보 처리 및 사각형 그리기
        if results.detections:
            for detection in results.detections:
                mp_drawing.draw_detection(image, detection)

        # 결과 이미지 표시
        cv2.imshow('Face Detection', image)

        # 'q'를 누르면 반복을 종료하고 창을 닫음
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# 모든 자원 해제
cap.release()
cv2.destroyAllWindows()


2.얼굴 전체 메시 그려보기

In [3]:
# MediaPipe 얼굴 메시 객체와 그리기 객체 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# 웹캠으로부터 비디오 캡처 시작
cap = cv2.VideoCapture(0)

# with 문으로 작성해보기
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue

        # BGR 이미지를 RGB로 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # 성능 향상을 위해 이미지 쓰기 불가능으로 설정
        image.flags.writeable = False
        
        # 이미지에서 얼굴 메시 검출
        results = face_mesh.process(image)

        # 이미지를 다시 BGR로 변환하여 그리기 작업 준비
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 검출된 얼굴 메시에 대한 정보 처리
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # 전체 얼굴 랜드마크 그리기
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=drawing_spec,
                    connection_drawing_spec=drawing_spec)
                
                # 특정 부위별로 랜드마크 그리기 (예: 눈, 입, 눈썹)
                # 여기서는 전체 얼굴 랜드마크를 그리고 있지만, 필요에 따라 특정 부위만 선택하여 그릴 수 있습니다.
                # 예를 들어, 눈의 랜드마크만 그리고 싶다면 눈에 해당하는 랜드마크 인덱스를 사용하여 그리면 됩니다.

        # 결과 이미지 표시
        cv2.imshow('Face Mesh', image)

        # 'q'를 누르면 반복을 종료하고 창을 닫음
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# 모든 자원 해제
cap.release()
cv2.destroyAllWindows()

2-2. 눈 부위의 랜드 마크 그려보기

In [4]:
# MediaPipe 얼굴 메시(face mesh) 솔루션과 그리기 유틸리티를 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 각 눈의 랜드마크 인덱스를 정의
LEFT_EYE_INDICES = [362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385, 384, 398]
RIGHT_EYE_INDICES = [33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161, 246]

# 웹캠 시작
cap = cv2.VideoCapture(0)

# FaceMesh 모델을 초기화. 
with mp_face_mesh.FaceMesh(
    max_num_faces=1, # 최대 얼굴 수
    refine_landmarks=True, # 각 부위별 특징을 세밀하게 추출할지 결정
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5) as face_mesh:
    
    # 비디오가 열려 있을 경우 프레임 계속 읽어오기
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue  # 프레임 읽기 실패 시 다음 프레임으로

        # MediaPipe는 RGB이미지 사용 - BGR에서 RGB로색상 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Face Mesh 처리 결과
        results = face_mesh.process(image)

        # 그리기 작업을 위해 이미지를 다시 BGR로 변환
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 얼굴 랜드마크가 확인될 경우, 
        if results.multi_face_landmarks:

            # 검출된 각 얼굴에 랜드마크 표시
            for face_landmarks in results.multi_face_landmarks:

                # 왼쪽 눈을 파란색으로 그리기
                for idx in LEFT_EYE_INDICES:
                    point = face_landmarks.landmark[idx]
                    x = int(point.x * image.shape[1])
                    y = int(point.y * image.shape[0])
                    cv2.circle(image, (x, y), 1, (255, 0, 0), -1)

                # 오른쪽 눈을 녹색으로 그리기
                for idx in RIGHT_EYE_INDICES:
                    point = face_landmarks.landmark[idx]
                    x = int(point.x * image.shape[1])
                    y = int(point.y * image.shape[0])
                    cv2.circle(image, (x, y), 1, (0, 255, 0), -1)

        # 최종 결과 표시
        cv2.imshow('Eyes Landmarks', image)

        # 'q' 키를 누르면 반복을 종료합니다.
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# 자원을 해제합니다.
cap.release()
cv2.destroyAllWindows()

In [5]:
mp_face_mesh.FACEMESH_RIGHT_EYE

frozenset({(7, 163),
           (33, 7),
           (33, 246),
           (144, 145),
           (145, 153),
           (153, 154),
           (154, 155),
           (155, 133),
           (157, 173),
           (158, 157),
           (159, 158),
           (160, 159),
           (161, 160),
           (163, 144),
           (173, 133),
           (246, 161)})

2-2. 눈 부위의 랜드 마크 그려보기2

In [6]:
# MediaPipe Face Mesh 모듈과 그리기 모듈 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 웹캠 설정
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue

        # BGR 이미지를 RGB로 변환
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Face Mesh 처리
        results = face_mesh.process(image_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # 왼쪽 눈 랜드마크 그리기
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_LEFT_EYE,  # 오른쪽 눈 랜드마크 연결
                    landmark_drawing_spec=None,  # 랜드마크 스타일 지정 (None이면 기본값 사용)
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)  # 연결선 스타일 지정
                )
                # 오른쪽 눈 랜드마크 그리기
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_RIGHT_EYE,  # 오른쪽 눈 랜드마크 연결
                    landmark_drawing_spec=None,  # 랜드마크 스타일 지정 (None이면 기본값 사용)
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)  # 연결선 스타일 지정
                )

        # 결과 이미지 표시
        cv2.imshow('Eyes Landmarks', image)

        # 'q'를 눌러 종료
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


2-3. 눈썹 부위의 랜드 마크 그려보기

In [7]:
# MediaPipe 얼굴 메시(face mesh) 솔루션과 그리기 유틸리티를 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 왼쪽 눈썹의 상세한 랜드마크 인덱스
LEFT_EYEBROW_INDICES = [70, 63, 105, 66, 107, 55, 65, 52, 53, 46, 124, 35, 107, 336]

# 오른쪽 눈썹의 상세한 랜드마크 인덱스
RIGHT_EYEBROW_INDICES = [300, 293, 334, 296, 336, 285, 295, 282, 283, 276, 353, 265, 340, 285]

# 웹캠 시작
cap = cv2.VideoCapture(0)

# FaceMesh 모델을 초기화. 
with mp_face_mesh.FaceMesh(
    max_num_faces=1, # 최대 얼굴 수
    refine_landmarks=True, # 각 부위별 특징을 세밀하게 추출할지 결정
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5) as face_mesh:
    
    # 비디오가 열려 있을 경우 프레임 계속 읽어오기
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue  # 프레임 읽기 실패 시 다음 프레임으로

        # MediaPipe는 RGB이미지 사용 - BGR에서 RGB로색상 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Face Mesh 처리 결과
        results = face_mesh.process(image)

        # 그리기 작업을 위해 이미지를 다시 BGR로 변환
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # 얼굴 랜드마크가 확인될 경우, 
        if results.multi_face_landmarks:

            # 검출된 각 얼굴에 랜드마크 표시
            for face_landmarks in results.multi_face_landmarks:

                # 왼쪽 눈썹을 파란색으로 그리기
                for idx in LEFT_EYEBROW_INDICES:
                    point = face_landmarks.landmark[idx]
                    x = int(point.x * image.shape[1])
                    y = int(point.y * image.shape[0])
                    cv2.circle(image, (x, y), 1, (255, 0, 0), -1)

                # 오른쪽 눈썹을 녹색으로 그리기
                for idx in RIGHT_EYEBROW_INDICES:
                    point = face_landmarks.landmark[idx]
                    x = int(point.x * image.shape[1])
                    y = int(point.y * image.shape[0])
                    cv2.circle(image, (x, y), 1, (0, 255, 0), -1)

        # 최종 결과 표시
        cv2.imshow('Eye Brows', image)

        # 'q' 키를 누르면 반복을 종료합니다.
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# 자원을 해제합니다.
cap.release()
cv2.destroyAllWindows()

2-3. 눈썹 부위의 랜드 마크 그려보기2

In [8]:
# MediaPipe Face Mesh 모듈과 그리기 모듈 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 웹캠 설정
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue

        # BGR 이미지를 RGB로 변환
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Face Mesh 처리
        results = face_mesh.process(image_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # 왼쪽 눈썹 랜드마크 그리기
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_LEFT_EYEBROW, # 왼쪽 눈썹 랜드마크 연결
                    landmark_drawing_spec=None,  # 랜드마크 스타일 지정 (None이면 기본값 사용)
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)  # 연결선 스타일 지정
                )
                # 오른쪽 눈썹 랜드마크 그리기
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_RIGHT_EYEBROW,  # 오른쪽 눈썹 랜드마크 연결
                    landmark_drawing_spec=None,  # 랜드마크 스타일 지정 (None이면 기본값 사용)
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)  # 연결선 스타일 지정
                )

        # 결과 이미지 표시
        cv2.imshow('Eye Brows', image)

        # 'q'를 눌러 종료
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

2-4. 코 부위의 랜드마크 그려보기

In [9]:
# MediaPipe Face Mesh 모듈과 그리기 모듈 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 웹캠 설정
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue

        # BGR 이미지를 RGB로 변환
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Face Mesh 처리
        results = face_mesh.process(image_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # 코 부위의 랜드마크 그리기
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_NOSE, 
                    landmark_drawing_spec=None,  # 랜드마크 스타일 지정 (None이면 기본값 사용)
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)  # 연결선 스타일 지정
                )

        # 결과 이미지 표시
        cv2.imshow('Nose', image)

        # 'q'를 눌러 종료
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

2-5. 입의 랜드 마크 그려보기

In [10]:
# MediaPipe Face Mesh 모듈과 그리기 모듈 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 웹캠 설정
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue

        # BGR 이미지를 RGB로 변환
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Face Mesh 처리
        results = face_mesh.process(image_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # 코 부위의 랜드마크 그리기
                mp_drawing.draw_landmarks(
                    image=image,
                    landmark_list=face_landmarks,
                    connections=mp_face_mesh.FACEMESH_LIPS, 
                    landmark_drawing_spec=None,  # 랜드마크 스타일 지정 (None이면 기본값 사용)
                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=1)  # 연결선 스타일 지정
                )

        # 결과 이미지 표시
        cv2.imshow('Lips', image)

        # 'q'를 눌러 종료
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

2-6. 얼굴만 인식해서 얼굴만 따오기 - 추가 내용

In [11]:
# MediaPipe 얼굴 메시와 그리기 객체 초기화
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# 웹캠으로부터 비디오 캡처 시작
cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
    
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue

        # BGR 이미지를 RGB로 변환
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # 성능 향상을 위해 이미지 쓰기 불가능으로 설정
        image.flags.writeable = False
        
        # 이미지에서 얼굴 메시 검출
        results = face_mesh.process(image)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # 얼굴 경계 찾기
                h, w, _ = image.shape
                xmin = w
                ymin = h
                xmax = ymax = 0
                for lm in face_landmarks.landmark:
                    x, y = int(lm.x * w), int(lm.y * h)
                    if x < xmin:
                        xmin = x
                    if x > xmax:
                        xmax = x
                    if y < ymin:
                        ymin = y
                    if y > ymax:
                        ymax = y
                
                # 얼굴 부위 추출
                face_image = image[ymin:ymax, xmin:xmax]
                # RGB 이미지를 BGR로 변환하여 표시 준비
                face_image_bgr = cv2.cvtColor(face_image, cv2.COLOR_RGB2BGR)
                cv2.imshow('Extracted Face', face_image_bgr)

        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# 모든 자원 해제
cap.release()
cv2.destroyAllWindows()